In [1]:
#!/usr/bin/python

#setup logging
import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s:\t%(message)s', level=logging.DEBUG, datefmt='%m-%d %H:%M:%S')
logger = logging.getLogger('simple_example')
#logger.setLevel(logging.DEBUG)

pdebug = lambda x: logger.debug(x)
pinfo = lambda x: logger.info(x)
perror = lambda x: logger.error(x)
pexception = lambda x: logger.critical(x)

In [2]:
import threading
import time
from queue import Queue

exitFlag = 0

class myThread (threading.Thread):
    def __init__(self, threadID, name, callback, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q
        self.callback = callback
        
    def run(self):
        pdebug("Starting " + self.name)
        process_data(self.name, self.q)
        #self.callback(self.name, 1,1)
        pdebug("Exiting " + self.name)
        
    
def process_data(threadName, q):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            data = q.get()
            queueLock.release()
            pinfo("%s processing %s" % (threadName, data))
        else:
            queueLock.release()
        time.sleep(1)

threadList = ["Thread-1", "Thread-2", "Thread-3"]
nameList = ["One", "Two", "Three", "Four", "Five"]
queueLock = threading.Lock()
workQueue = Queue(maxsize=10)
threads = []
threadID = 1

# Create new threads
for tName in threadList:
    thread = myThread(threadID, tName, process_data, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1

# Fill the queue
queueLock.acquire()
for word in nameList:
    workQueue.put(word)
queueLock.release()

# Wait for queue to empty
while not workQueue.empty():
    pass

# Notify threads it's time to exit
exitFlag = 1

# Wait for all threads to complete
for t in threads:
    t.join()
pinfo("Exiting Main Thread")

02-17 20:16:24:DEBUG:	Starting Thread-1
02-17 20:16:24:DEBUG:	Starting Thread-2
02-17 20:16:24:DEBUG:	Starting Thread-3
02-17 20:16:24:INFO:	Thread-2 processing One
02-17 20:16:24:INFO:	Thread-3 processing Two
02-17 20:16:25:INFO:	Thread-1 processing Three
02-17 20:16:25:INFO:	Thread-2 processing Four
02-17 20:16:25:INFO:	Thread-3 processing Five
02-17 20:16:26:DEBUG:	Exiting Thread-1
02-17 20:16:26:DEBUG:	Exiting Thread-2
02-17 20:16:26:DEBUG:	Exiting Thread-3
02-17 20:16:26:INFO:	Exiting Main Thread


In [2]:
import threading
import time
from queue import Queue
import redis
import multiprocessing

exitFlag = 0

conn = redis.Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

class myThread (threading.Thread):
    def __init__(self, threadID, name, callback):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.callback = callback
        
    def run(self):
        pdebug("Starting " + self.name)
        self.thread_function(self.callback)
        pdebug("Exiting " + self.name)
        
    def thread_function(self, callback):
        pubsub = conn.pubsub()
        pubsub.subscribe([self.name+'/cmd', self.name+'/data'])

        for item in pubsub.listen():
            channel = item['channel'].split('/')[1]
            data = item['data']
            pdebug(self.name+':'+channel)
            if channel== 'data':
                callback(data)
            elif channel== 'cmd' and data == 'stop':
                pubsub.unsubscribe()
                break
jobs = []
class threadManager():
    def __init__(self, name, thread_list, callback_list):
        self.threads = []
        self.name = name
        self.threadList = thread_list
        self.threadCallback = callback_list
        self.threadID = 1
        
        self.job = multiprocessing.Process(target=self.init)
        jobs.append(self.job)
        self.job.start()
        
    def init(self):
        # Create new threads
        for tName in self.threadList:
            self.add(tName, self.threadCallback[self.threadID-1])

        # Wait for all threads to complete
        #for t in self.threads:
        #    t.join()
        #pinfo("Exiting Main Thread")
        
    def add(self, name, callback):
        thread = myThread(self.threadID, name, callback)
        thread.start()
        self.threads.append(thread)
        self.threadID += 1

In [3]:
def hello_world1(data):
    pdebug("1: "+ str(data))
    
def hello_world2(data):
    pdebug("2: "+ str(data))
    

tradeManager = threadManager("tradeManager", ["order", "trade"], [hello_world1, hello_world2])


02-17 20:19:42:DEBUG:	Starting order
02-17 20:19:42:DEBUG:	Starting trade
02-17 20:19:42:DEBUG:	order:cmd
02-17 20:19:42:DEBUG:	order:data
02-17 20:19:42:DEBUG:	trade:cmd
02-17 20:19:42:DEBUG:	trade:data
02-17 20:19:42:DEBUG:	2: 2
02-17 20:19:42:DEBUG:	1: 2
02-17 20:20:09:DEBUG:	trade:cmd
02-17 20:20:09:DEBUG:	Exiting trade
02-17 20:20:13:DEBUG:	order:cmd
02-17 20:20:13:DEBUG:	Exiting order


In [4]:
tradeManager.add("log",hello_world1)

02-17 20:19:45:DEBUG:	Starting log
02-17 20:19:45:DEBUG:	log:cmd
02-17 20:19:45:DEBUG:	log:data
02-17 20:19:45:DEBUG:	1: 2
02-17 20:19:59:DEBUG:	log:data
02-17 20:19:59:DEBUG:	1: stop
02-17 20:20:20:DEBUG:	log:cmd
02-17 20:20:20:DEBUG:	Exiting log


In [5]:
jobs[0].is_alive()

False